# INSTALANDO DEPENDÊNCIAS

In [ ]:
# Instalando módulos novos
! pip install numpy
! pip install sklearn
! pip install tensorflow
! pip install tflearn
#! pip install pypiwin32
! pip install scrapy

In [ ]:
# Importações Iniciais
import os
import nltk
import time
import random
import numpy as np
import tensorflow as tf
import tflearn
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split 


In [ ]:
# definição de variaveis 
FILES_DIR = 'news'
FILES_DIR_UNKNOW = 'desconhecidos'
MINIMO_LETRAS = 3
PALAVRAS_IGNORADAS = ['@', '.', '!','?',',','$','-','\'s','g','(',')','[',']',':','\n']
NUMERO_PALAVRAS_MAIS_FREQUENTES = 1000
PERCENTUAL_TESTE = 0.33
PASSOS_TREINAMENTO = 100
BATCH_SIZE = 50
VALIDATION_SET = 0.1

# IMPORTANDO DATASET E NOTICIAS

In [ ]:
start = time.time()
categorias = []
documentos = []
resultados = []
i = 0
for _, dirs, _ in os.walk(os.path.join(FILES_DIR)):
    for d in dirs:
        if d not in categorias:
            categorias.append(d)
        print('lendo categoria {}'.format(d))
        for _, _, files in os.walk(os.path.join(FILES_DIR,d)):
            for f in files:
                i += 1
                if i % 100 == 0:
                    print('{} arquivos lidos. lendo {}'.format(i,f))
                with codecs.open(os.path.join(FILES_DIR,d,f),'r',encoding="utf8", errors='ignore') as f:                     
                     documentos.append(f.read())
                resultados.append(d)
end = time.time()                
print ('\nExistem {} textos e {} categorias: {}. \nProcessamento em {}s'.format(len(documentos),len(categorias),categorias,end - start))                                

## FORMATAÇÃO DOS INPUTS

## Tokenização

In [ ]:
start = time.time()
documentos_tokenizados = []
for documento in documentos:
    documentos_tokenizados.append(word_tokenize(documento))    
end = time.time()
print ('{} documentos tokenizados. \nProcessamento em {}s'.format(len(documentos_tokenizados),end - start))                                

In [ ]:
print(documentos_tokenizados[0])

## Stop Words

In [ ]:
start = time.time()
ptbr_stops = set(stopwords.words('portuguese'))
documentos_sem_stop_words_e_ignore = []
for documento in documentos_tokenizados:    
    aux = [w for w in documento if w not in ptbr_stops]   
    aux2 = [w for w in aux if w not in PALAVRAS_IGNORADAS]    
    documentos_sem_stop_words_e_ignore.append([w for w in aux2 if len(w) >= MINIMO_LETRAS])
end = time.time()
print ('{} documentos removidos as stop words e palavras ignoradas. \nProcessamento em {}s'
       .format(len(documentos_sem_stop_words_e_ignore),end - start))

In [ ]:
print(documentos_sem_stop_words_e_ignore[0])

## Stemming

In [ ]:
# Processando o stemming para cada palavra, transformando em minísculo
start = time.time()
stemmer = SnowballStemmer('portuguese')
documentos_apos_stemming = []
for documento in documentos_sem_stop_words_e_ignore:
    # List comprehension para fazer o stemming de cada palavra
    documentos_apos_stemming.append([stemmer.stem(w.lower()) for w in documento])
end = time.time()
print ('{} documentos após aplicar stemmer. \nProcessamento em {}s'.format(len(documentos_apos_stemming),end - start))   

In [ ]:
# Imprimindo o primeiro documento após processamento
print(documentos_apos_stemming[0])

## Frequência

In [ ]:
start = time.time()
todas_palavras = []
for documento in documentos_apos_stemming:
    todas_palavras.extend(documento)
end = time.time()
print ('foram reunidas um todal de {} palavras. \nProcessamento em {}s'.format(len(todas_palavras),end - start))

In [ ]:
start = time.time()
fdist = nltk.FreqDist(todas_palavras)
palavras_mais_frequentes = []
for word_freq in fdist.most_common(NUMERO_PALAVRAS_MAIS_FREQUENTES):
    palavras_mais_frequentes.append(word_freq[0])
palavras_mais_frequentes = sorted(list(set(palavras_mais_frequentes)))
end = time.time()
print('Número total de tokens distintos {}. \nSelecionando apenas as {} palavras mais frequentes. \nProcessamento em {}s'
      .format(fdist.N(),NUMERO_PALAVRAS_MAIS_FREQUENTES,end - start))

In [ ]:
print(len(palavras_mais_frequentes[0]))

# Bag of Words

In [ ]:
start = time.time()
documentos_bow = []
resultados_bow = []
for i in range(len(documentos_apos_stemming)):
    documento = documentos_apos_stemming[i]
    resultado = resultados[i]   
    documento_bow = []
    # Cria um array com o bag of words de N palavras mais frequentes, colocando 1 quando a palavra estiver presente no documento
    for w in palavras_mais_frequentes:
        documento_bow.append(1) if w in documento else documento_bow.append(0)
    documentos_bow.append(documento_bow)
    # Inicializa o array de resultados com zero    
    resultado_bow = list([0] * len(categorias))
    # Coloca 1 na posição do array correta de acordo com o resultado   
    resultado_bow[categorias.index(resultado)] = 1
    resultados_bow.append(resultado_bow)
end = time.time()
print('Criou um total de {} documentos BoW. \nProcessamento em {}s'.format(len(documentos_bow),end - start))

In [ ]:
# Imprimindo o primeiro documento BoW
print('primeiro documento {}'.format(documentos_bow[0]))

In [ ]:
# Imprimindo o primeiro documento BoW
print(len(resultados_bow))
print('primeiro resultado BoW {} para as categorias {}'.format(resultados_bow[1], categorias))

## NUMPY

In [ ]:
documentos_bow = np.array(documentos_bow)
resultados_bow = np.array(resultados_bow)

## TESTE VS TREINAMENTO

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(documentos_bow, resultados_bow, test_size=PERCENTUAL_TESTE, random_state=42)
print('Separando {} documentos/resultados para treinamento e {} para teste'.format(len(X_train),len(X_test)))
print(len(X_train[0]))

## NOSSA REDE

In [ ]:
tf.reset_default_graph()
net = tflearn.input_data(shape=[None, len(X_train[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
net = tflearn.regression(net)
# Define o modelo e configura o tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs/'+time.strftime("%Y%m%d_%H%M"))
print('Modelo de rede neural criado: {}'.format(model))

## Realizando o Treinamento

In [ ]:
model.fit(X_train, y_train, n_epoch=PASSOS_TREINAMENTO, batch_size=BATCH_SIZE, validation_set=VALIDATION_SET, show_metric=True)

## ACURÁCIA

In [ ]:
score = model.evaluate(X_test, y_test)
print('Test accuarcy: %0.4f%%' % (score[0] * 100))

In [ ]:
print(len(X_test))
print(len(y_test))
prediction = model.predict([X_test[4]])
print("Valor previsto: {}. \nValor esperado: {}".format(prediction[0],y_test[1]))

In [ ]:
## VALIDANDO NA PRÁTICA

In [ ]:
nova = ''
nova_token = []
nova_no_stop = []
nova_stemming = []
novas_bow = []
with open(os.path.join(FILES_DIR_UNKNOW,'noticia1.txt'),'r',encoding="utf8", errors='ignore') as f: 
                  nova = f.read()
                   
#------------------------------------------lido nova------------------------------------------------'         

nova_token=word_tokenize(nova)

#'--------------------------------------------token---------------------------------------------------'


aux = [w for w in nova_token if w not in ptbr_stops]   
aux2 = [w for w in aux if w not in PALAVRAS_IGNORADAS]    
nova_no_stop.append([w for w in aux2 if len(w) >= MINIMO_LETRAS])

#'-------------------------------------------steammin------------------------------------------------'

nova_stemming.append([stemmer.stem(w.lower()) for w in nova_no_stop[0]])


#'-------------------------------------------bag-------------------------------------------------------'

for w in palavras_mais_frequentes:
        novas_bow.append(1) if w in nova_stemming[0] else novas_bow.append(0)
    
prediction = model.predict([X_test[1]])
print("Valor previsto: {}. \nValor esperado: {}".format(prediction[0],1))